<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Scraping/1_HTTP_Inicial/ejercicio/7Puentes_ejercicios_web_scraping_http_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

# **Web scraping con Python**

## Más ejercicios!
<img src="https://c.tenor.com/S4Sz_yvlLn4AAAAC/cats-cat.gif" height=250>


## **Vamos a scrapear las novedades de Goodreads.com**
<hr>
<img src='https://digital.hbs.edu/platform-digit/wp-content/uploads/sites/2/2015/10/goodreads-620x200.png'>

- GoodReads es una red social donde se comparte lo que uno esta leyendo, se ofecen recomendaciones de lectura, etc.

- En particular nos ofrece una serie de titulos catalogados como "novedades" para varios generos entrando a: https://www.goodreads.com/genres/


El objetivo del ejercicios será *scrapear* esas novedades para un género de nuestro interés (o para todos!)

1. Obtener el HTML de https://www.goodreads.com/genres/new_releases/{CATEGORIA}, siendo CATEGORIA el género de interés
2. Entrar a cada uno de los links de los libros
3. Obtener el título y el resumen
4. Guardar toda la informacion en un archivo *.csv* 

In [1]:
# Importamos librerias
from bs4 import BeautifulSoup
import requests
import json
import time
import re 
import pandas as pd

In [2]:
def get_title_resume(url_categoria):
    url_basis = 'https://www.goodreads.com/book/show'

    id_libros_container = []
    book_names_resumes = []

    headers = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }
    
    html_obtained = requests.get(url_categoria, headers=headers)
    soup = BeautifulSoup(html_obtained.text, "html.parser")
    books_container = soup.find_all('img', class_='bookImage')

    for book in books_container:
        id_libros_container.append(book['src'].split('/')[-1].split('.')[0])

    for id in id_libros_container:
        html_obtained_libros = requests.get(f'{url_basis}/{id}', headers=headers)
        soup_infos_libros = BeautifulSoup(html_obtained_libros.text, "html.parser")
        book_name = soup_infos_libros.find_all('h1', {'data-testid':'bookTitle'})
        book_name = book_name[0].string.strip()
        book_resume = soup_infos_libros.find_all('div', {'data-testid':'description'})
        book_resume = book_resume[0].text.strip()
        book_names_resumes.append((book_name,book_resume))

    df_books = pd.DataFrame(book_names_resumes, columns=['Book title','Book resume'])

    return df_books

In [12]:
get_title_resume('https://www.goodreads.com/genres/new_releases/poetry')

,Book title,Book resume
0,Everything Is Poison,A Blood Water Paint-style historical YA in pro...
1,"Helen of Troy, 1993: Poems","Part myth retelling, part character study, thi..."
2,Safe Harbor,An uplifting novel in verse about an immigrant...
3,At the End of the World There Is a Pond: Poems,A stunning debut volume infused with apocalypt...
4,The Wickedest,'Atmospheric and intoxicating … The Wickedest ...
5,Stars Around My Scars: The Annotated Poetry of...,Grab your highlighters and favorite red pen an...
6,All the Love Under the Vast Sky,Twelve short stories in verse by bestselling a...
7,"Soul Sessions: Haikus on Love, Liberation, Her...","Soul Sessions: Haikus on Love, My Heritage, Li..."
8,An Absence of Fear,An Absence of Fear is a brave book by a poet w...
9,Invisible Strings: 113 Poets Respond to the So...,An anthology of brand-new poems inspired by Ta...


De aquí en adelante se realizaron pruebas para luego crear la función de arriba

In [24]:
url_basis = 'https://www.goodreads.com/book/show'
url_categoria = 'https://www.goodreads.com/genres/new_releases/economics'
book_url = []

In [4]:
headers = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }

In [5]:
html_obtained = requests.get(url_categoria, headers=headers)

In [6]:
html_obtained

<Response [200]>

In [7]:
soup = BeautifulSoup(html_obtained.text, "html.parser")

In [8]:
books_container = soup.find_all('img', class_='bookImage')

In [9]:
id_libros_container = []

In [10]:
# Función para obtener el enlace de la portada del libro

for book in books_container:
    # print(book['href'])
    id_libros_container.append(book['src'].split('/')[-1].split('.')[0])

In [11]:
id_libros_container

['52979663', '55529461']

In [12]:
book_names_resumes = []

In [13]:
for id in id_libros_container:
    html_obtained_libros = requests.get(f'{url_basis}/{id}', headers=headers)
    soup_infos_libros = BeautifulSoup(html_obtained_libros.text, "html.parser")
    book_name = soup_infos_libros.find_all('h1', {'data-testid':'bookTitle'})
    book_name = book_name[0].string.strip()
    book_resume = soup_infos_libros.find_all('div', {'data-testid':'description'})
    book_resume = book_resume[0].text.strip()
    book_names_resumes.append((book_name,book_resume))

In [15]:
df_books = pd.DataFrame(book_names_resumes, columns=['Book title','Book resume'])

In [16]:
df_books

,Book title,Book resume
0,Change Everything: Racial Capitalism and the C...,"Racial, gender, and environmental justice. Cla..."
1,Breaking Things at Work: The Luddites Are Righ...,"In the nineteenth century, English textile wor..."


In [5]:
test_url = requests.get("https://www.goodreads.com/book/show/52979663")

In [6]:
test_soup = BeautifulSoup(test_url.text, "html.parser")

In [7]:
test_soup

<!DOCTYPE html>
<html lang="en"><head><script>var ue_t0=window.ue_t0||+new Date();(function(e){var c=e,a={main_scope:"mainscopecsm",q:[],t0:c.ue_t0||+new Date(),d:g};function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(k,j,h){var i={m:k,f:j,l:h,fromOnError:1,args:arguments};c.ueLogError(i);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElementsByTagName("head")[0]||d.documentElement,h=d.createElement("script");h.async="async";h.src=g;f.insertBefore(h,f.firstChild)}function b(){var k=e.ue_cdn||"z-ecx.images-amazon.com",g=e.ue_cdns||"images-na.ssl-images-amazon.com",j=

In [91]:
book_name = test_soup.find_all('h1', {'data-testid':'bookTitle'})

In [105]:
book_name = book_name[0].string.strip()

In [106]:
book_name

'Change Everything: Racial Capitalism and the Case for Abolition'

In [129]:
book_resume = test_soup.find_all('div', {'data-testid':'description'})

In [131]:
book_resume = book_resume[0].text.strip()

In [132]:
book_resume

"Racial, gender, and environmental justice. Class war. Militarism. Interpersonal violence. Old age security. This is not the vocabulary many use to critique the prison-industrial complex. But in this series of powerful lectures, Ruth Wilson Gilmore shows that the only way to dismantle systems and logics of control and punishment is to change questions, categories, and campaigns from the ground up.Abolitionism doesn't just say no to police, prisons, border control, and the current punishment system. It requires persistent organizing for what we need, organizing that's already present in the efforts people cobble together to achieve access to schools, health care and housing, art and meaningful work, and freedom from violence and want. As Gilmore makes plain, Abolition requires that we change one thing: everything.Change Everything is the inaugural book in the new Abolitionist Papers book series, edited by Naomi Murakawa."

In [67]:
soup

<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head>
<title>Economics Books</title>
<meta content="Economics genre: new releases and popular books, including Change Everything: Racial Capitalism and the Case for Abolition by Ruth Wilson Gilmore und Bre..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/genres/economics" rel="canonical"/>
<meta content="noindex" name="robots"/>
<!-- * Copied from https://info.analytics.a2z.com/#/docs/data_collection/csa/onboard */ -->
<script>
  //<![CDATA[
    !function(){function n(n,t){var r=i(n);return t&&(r=r("instance",t)),r}var r=[],c=0,i=function(t){return function(){var n=c++;return r.push([t,[].slice.call(arguments,0),n,{time:Date.now()}]),i(n)}};n._s=r,this.csa=n}();
    
    if (window.csa) {
      window.csa("Config", {
        "Application": "GoodreadsMonolith",
        "Events.SushiEndpoint": "https://unagi.amazon.com/1/events/com.amazon.csm.csa.prod",

Documentacion de nuestra función: 
 
  Scraper de libros de resumenes de libros que aparecen como 'New realeases' en GoodReads
 
  parameters:
            url_category  - String conteniendo una URL del tipo ' https://www.goodreads.com/genres/new_releases/{CATEGORIA} ' 
            base_url - String conteniendo la URL base de Goodreads. Actualmente: 'https://www.goodreads.com'
 
  return:
            titulos       - Lista de strings con los titulos de los libros correspondientes a la categoria 
            resumenes     - Lista de strings con los resumenes de los libros correspondientes a la categoria 
            df            - Pandas Dataframe cuyos índices son los títulos de los libros y cuya única columna contiene los resúmenes
 
   

URL de categoria: https://www.goodreads.com/genres/new_releases/Biography 
Status code: 200
URL de libro: https://www.goodreads.com/book/show/58558175-ancestor-trouble 
Status code: 200
El titulo es: 'Ancestor Trouble: A Reckoning and a Reconciliation

,Resumenes
Ancestor Trouble: A Reckoning and a Reconciliation,An acclaimed writer goes searching for the tru...
"The Other Dr. Gilmer: Two Men, a Murder, and an Unlikely Fight for Justice",A rural physician learns that a former doctor ...
Comedy Comedy Comedy Drama,"In this hilarious, heartfelt memoir, the star ..."
In Love: A Memoir of Love and Loss,This powerful memoir by New York Times bestsel...
No hay titulo,Multi-awardwinning Hannah Gadsby transformed c...
Ten Steps to Nanette,"From Helen Rappaport, the New York Times bests..."
After the Romanovs: Russian Exiles in Paris from the Belle Époque Through Revolution and War,"A beautifully written, witty memoir that is al..."
"Every Good Boy Does Fine: A Love Story, in Music Lessons",A landmark exploration of one of the most cons...
The Invisible Kingdom: Reimagining Chronic Illness,AN INSTANT NATIONAL BESTSELLERMemoir meets cra...
Body Work: The Radical Power of Personal Narrative,A poignant and hilarious memoir from the cultu...


 
 Resumen de 'Ancestor Trouble: A Reckoning and a Reconciliation':  



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Si se coparon mucho pueden hacer un mega scrapeo iterando sobre algunas de las categorias
 
### **Tip para que no les raje colab:**
* En la consola del navegador pegan esto y le dan enter:



```javascript

// Código en JavaScript
function ClickConnect() {
  console.log('Trabajando...')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
intervalTiming = setInterval(ClickConnect, 60000)
```
 
Para desactivarlo: 
 
```javascript
clearInterval(intervalTiming)
```
 
 
 
 
 
<img src='https://bookbub-res.cloudinary.com/image/upload/f_auto,q_auto/v1584035706/blog/justification-for-too-many-books-memes-we-need-more-books.jpg'>

Art
URL de categoria: https://www.goodreads.com/genres/new_releases/Art 
Status code: 200
URL de libro: https://www.goodreads.com/book/show/58275730-woman-on-fire 
Status code: 200
El titulo es: 'Woman on Fire' 

Resumen: From the author of the award-winning Fugitive Colors and The Unbreakables, a gripping tale of a young, ambitious journalist embroiled in an international art art scandal centered around a Nazi-looted masterpiece--forcing the ultimate showdown between passion and possession, lovers and liars, history and truth. NOW A NEW YORK TIMES BESTSELLER! Actress Sharon Stone set to produce and star in the film adaptation of WOMAN ON FIRE. After talking her way into a job with Dan Mansfield, the leading investigative reporter in Chicago, rising young journalist Jules Roth is given an unusual--and very secret--assignment. Dan needs her to locate a painting stolen by the Nazis more than 75 years earlier: legendary Expressionist artist Ernst Engel's most famous work, Woman on Fire. W

In [ ]:
diccionario_de_dataframes['Biography']

,Resumenes
Ancestor Trouble: A Reckoning and a Reconciliation,An acclaimed writer goes searching for the tru...
No hay titulo,An Indigenous artist blends the aesthetics of ...
Comedy Comedy Comedy Drama,This powerful memoir by New York Times bestsel...
In Love: A Memoir of Love and Loss,Liz Scheier’s darkly funny and touching memoir...
Never Simple,Multi-awardwinning Hannah Gadsby transformed c...
Ten Steps to Nanette,"From Helen Rappaport, the New York Times bests..."
After the Romanovs: Russian Exiles in Paris from the Belle Époque Through Revolution and War,"A beautifully written, witty memoir that is al..."
"Every Good Boy Does Fine: A Love Story, in Music Lessons",A landmark exploration of one of the most cons...
The Invisible Kingdom: Reimagining Chronic Illness,AN INSTANT NATIONAL BESTSELLERMemoir meets cra...
Body Work: The Radical Power of Personal Narrative,All Kait Leddy had ever wanted was a little si...
